In [ ]:
# CODE FOR LOADING SAVED CHATBOT
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import speech_recognition as sr
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob


# load intents and saved model output files
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

# ... [Your existing setup code] ...

# Initialize a dictionary for session storage
conversation_history = {}

# Function to handle a chat message
def handle_chat(session_id, user_input):
    if session_id not in conversation_history:
        conversation_history[session_id] = []

    # Existing chatbot processing logic
    ints = predict_class(user_input)
    res = get_response(ints, intents)

    # Update the conversation history
    conversation_history[session_id].append({"user": user_input, "bot": res})

    # Sentiment analysis and additional processing
    sentiment = determine_sentiment(user_input)
    print(f"Sentiment: {sentiment}")
    if sentiment == 'negative':
        with open('negative_conversations.txt', 'a') as f:
            f.write(f"Bot: {res}\n")
            f.write(f"User: {user_input}\n")
            f.write("---\n")

    return res

# End a chat session
def end_session(session_id):
    if session_id in conversation_history:
        del conversation_history[session_id]

# Main loop for chat interaction
print("Chatbot live. Say or type 'end' to stop the conversation.")
session_id = "user123"  # Replace with dynamic session ID logic as needed
while True:
    user_input = chat_input_method()
    if user_input.lower() == "end":
        end_session(session_id)
        print("Bot: Goodbye! Have a nice day!")
        break
    elif user_input != "":
        bot_response = handle_chat(session_id, user_input)
        print("Bot:", bot_response)


In [2]:
# CODE FOR LOADING SAVED CHATBOT
# import libraries
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import speech_recognition as sr
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob


# load intents and saved model output files
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

# separate input words from input sentence
def clean_up_sentences(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
	return sentence_words

# determine if input words in saved model words list
def bagw(sentence):
	sentence_words = clean_up_sentences(sentence)
	bag = [0]*len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

# predict label of sentence based on input words
# update: leveraging list comprehensions and dictionaries to work with confidence scores
def predict_class(sentence):
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [{'intent': labels[i], 'probability': str(res[i])}
               for i in range(len(res)) if res[i] > ERROR_THRESHOLD]
    results.sort(key=lambda x: x['probability'], reverse=True)
    return results

# print random response from appropriate responses for label
# if predicted intent has a low confidence score, gives a secondary response
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    probability = float(intents_list[0]['probability'])
    list_of_intents = intents_json['intents']
    result = ""
    if probability < 0.2:  # Adjust the threshold as needed
        result = "I'm sorry, but I'm not sure how to help with that. Could you please provide more information?"
    else:
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    return result

# Capture the user's speech input
def listen_to_user():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = r.listen(source, timeout=10, phrase_time_limit=10)
            voice_input = r.recognize_google(audio)
            print("You said: {}".format(voice_input))
            return voice_input
        except sr.WaitTimeoutError:
            print("No command received in 10 seconds.")
            return ""
        except sr.UnknownValueError:
            print("Sorry, I didn't get that. Could you say it again?")
            return listen_to_user()
        except sr.RequestError:
            print("Sorry, my speech service is down. Please try again later.")
            return ""
        
# Initialize a variable to store the last bot reply
last_bot_reply = ""

# Initialize counts and list to store negative conversations
positive_count = 0
negative_count = 0
neutral_count = 0
total_count = 0
negative_conversations = []

# Determine sentiment of user's reply
def determine_sentiment(message):
    global positive_count, negative_count, neutral_count, total_count
    analysis = TextBlob(message)
    total_count += 1
    sentiment = "neutral"
    
    if analysis.sentiment.polarity > 0:
        positive_count += 1
        sentiment = 'positive'
    elif analysis.sentiment.polarity < 0:
        negative_count += 1
        sentiment = 'negative'
    else:
        neutral_count += 1
    
    return sentiment


# Determine user input method
def chat_input_method():
    print("How would you like to communicate with the bot?")
    print("1. Type\n2. Speak")
    method = input("Enter the number associated with your preferred method: ")
    
    if method == "1":
        user_input = input("You: ")
        print("You said: ", user_input)
        return user_input
    elif method == "2":
        return listen_to_user()
    else:
        print("Invalid input. Please enter 1 for Typing or 2 for Speaking.")
        return chat_input_method()
    
    
# Function to handle a chat message
def handle_chat(session_id, user_input):
    if session_id not in conversation_history:
        conversation_history[session_id] = []

    # Existing chatbot processing logic
    ints = predict_class(user_input)
    res = get_response(ints, intents)

    # Update the conversation history
    conversation_history[session_id].append({"user": user_input, "bot": res})

    # Sentiment analysis and additional processing
    sentiment = determine_sentiment(user_input)
    print(f"Sentiment: {sentiment}")
    if sentiment == 'negative':
        with open('negative_conversations.txt', 'a') as f:
            f.write(f"Bot: {res}\n")
            f.write(f"User: {user_input}\n")
            f.write("---\n")

    return res

# End a chat session
def end_session(session_id):
    if session_id in conversation_history:
        del conversation_history[session_id]
        

# Initialize a dictionary for session storage
conversation_history = {}
        
# Main loop for chat interaction
print("Chatbot live. Say or type 'end' to stop the conversation.")
session_id = "user123"  # Replace with dynamic session ID logic as needed

# Get the communication method only once at the start of the session
print("How would you like to communicate with the bot?")
print("1. Type\n2. Speak")
communication_method = input("Enter the number associated with your preferred method: ")

while True:
    if communication_method == "1":
        user_input = input("You: ")
    elif communication_method == "2":
        user_input = listen_to_user()
    else:
        print("Invalid input. Please restart the chat and select a valid communication method.")
        break

    if user_input.lower() == "end":
        end_session(session_id)
        print("Bot: Goodbye! Have a nice day!")
        break
    elif user_input != "":
        bot_response = handle_chat(session_id, user_input)
        print("Bot:", bot_response)


C:\Users\adminguest.RUT-NORT-NB111\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\adminguest.RUT-NORT-NB111\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\adminguest.RUT-NORT-NB111\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to C:\Users\adminguest.RUT-NORT-
[nltk_data]     NB111\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\adminguest.RUT-
[nltk_data]     NORT-NB111\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Chatbot live. Say or type 'end' to stop the conversation.
How would you like to communicate with the bot?
1. Type
2. Speak
Enter the number associated with your preferred method: 1
You: good mornign
1/1 [==============================] - 0s 99ms/step
Sentiment: positive
Bot: Based on your previous purchases, you might like product Y.
You: good morning
1/1 [==============================] - 0s 26ms/step
Sentiment: positive
Bot: Based on your previous transaction you may save upto
You: hi how are you
1/1 [==============================] - 0s 25ms/step
Sentiment: neutral
Bot: You're welcome!
You: how are you
1/1 [==============================] - 0s 22ms/step
Sentiment: neutral
Bot: No problem!
You: can you help me
1/1 [==============================] - 0s 46ms/step
Sentiment: neutral
Bot: No problem!
You: how much balance do i have
1/1 [==============================] - 0s 15ms/step


IndexError: list index out of range

In [3]:
print(conversation_history[user123])


NameError: name 'user123' is not defined